# __Aggregating DataFrames__

# Outline
- [1 Summary Statistics](#sum-stats)
- [&nbsp;&nbsp;1.1 Mean and Median](#mean-med)
- [&nbsp;&nbsp;1.2 Summarizing Dates](#sum-dates)
- [&nbsp;&nbsp;1.3 Efficient Summaries](#sum-eff)
- [&nbsp;&nbsp;1.4 Cumulative Statistics](#cum-stats)
- [2 Counting](#count)
- [&nbsp;&nbsp;2.1 Dropping Duplicates](#drop-dup)
- [&nbsp;&nbsp;2.2 Counting Categorical Variables](#count-categ)
- [3 Grouped Summary Statistics](#grp-stat)
- [&nbsp;&nbsp;3.1 What percent of sales occurred at each store type?](#percent)
- [&nbsp;&nbsp;3.2 Calculations with .groupby()](#grp)
- [&nbsp;&nbsp;3.3 Multiple grouped summaries](#multi)
- [4 Pivot Tables](#pivo)
- [&nbsp;&nbsp;4.1 Pivoting on one variable](#pivo-one)
- [&nbsp;&nbsp;4.2 Fill in missing values and sum values with pivot tables](#fill)


Importing `pandas` & loading data into `sales`

In [2]:
import pandas as pd
sales = pd.read_csv("./../../data/sales_subset.csv", index_col=0)

<a id="sum-stats"></a>
# 1 __Summary Statistics__
<a id="mean-med"></a>
## 1.1 Mean and median<br>
Summary statistics are exactly what they sound like - they summarize many numbers in one statistic. For example, mean, median, minimum, maximum, and standard deviation are summary statistics. Calculating summary statistics allows you to get a better sense of your data, even if there's a lot of it.

Explore your new DataFrame first by printing the first few rows of the sales DataFrame.

In [3]:
sales.head()

,store,type,department,date,weekly_sales,is_holiday,temperature_c,fuel_price_usd_per_l,unemployment
0,1,A,1,2010-02-05,24924.50,False,5.727778,0.679451,8.106
1,1,A,1,2010-03-05,21827.90,False,8.055556,0.693452,8.106
2,1,A,1,2010-04-02,57258.43,False,16.816667,0.718284,7.808
3,1,A,1,2010-05-07,17413.94,False,22.527778,0.748928,7.808
4,1,A,1,2010-06-04,17558.09,False,27.050000,0.714586,7.808


Print information about the columns in sales.

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10774 entries, 0 to 10773
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   store                 10774 non-null  int64  
 1   type                  10774 non-null  object 
 2   department            10774 non-null  int64  
 3   date                  10774 non-null  object 
 4   weekly_sales          10774 non-null  float64
 5   is_holiday            10774 non-null  bool   
 6   temperature_c         10774 non-null  float64
 7   fuel_price_usd_per_l  10774 non-null  float64
 8   unemployment          10774 non-null  float64
dtypes: bool(1), float64(4), int64(2), object(2)
memory usage: 768.1+ KB


Print the mean of the weekly_sales column.

In [5]:
sales["weekly_sales"].mean()

23843.95014850566

Print the median of the weekly_sales column.

In [6]:
sales["weekly_sales"].median()

12049.064999999999

<a id="sum-dates"></a>
## 1.2 Summarizing dates<br>
Summary statistics can also be calculated on date columns that have values with the data type `datetime64`. Some summary statistics — like mean — don't make a ton of sense on dates, but others are super helpful, for example, minimum and maximum, which allow you to see what time range your data covers.

Print the maximum of the date column.

In [7]:
sales["date"].max()

'2012-10-26'

Print the minimum of the date column.

In [8]:
sales["date"].min()

'2010-02-05'

<a id="sum-eff"></a>
## 1.3 Efficient summaries<br>
While pandas and NumPy have tons of functions, sometimes, you may need a different function to summarize your data.

The `.agg()` method allows you to apply your own custom functions to a DataFrame, as well as apply functions to more than one column of a DataFrame at once, making your aggregations super-efficient. For example,<br>
`df['column'].agg(function)` <br>
In the custom function for this exercise, "IQR" is short for inter-quartile range, which is the 75th percentile minus the 25th percentile. It's an alternative to standard deviation that is helpful if your data contains outliers.

In [9]:
import numpy as np

def iqr(column):
    return column.quantile(0.75) - column.quantile(0.25)

Use the custom `iqr` function defined for you along with `.agg()` to print the IQR of the `temperature_c` column of `sales`.

In [10]:
sales["temperature_c"].agg(iqr)

16.583333333333336

Update the column selection to use the custom `iqr` function with `.agg()` to print the IQR of `temperature_c`, `fuel_price_usd_per_l`, and `unemployment`, in that order.

In [11]:
sales[["temperature_c", "fuel_price_usd_per_l", "unemployment"]].agg(iqr)

temperature_c           16.583333
fuel_price_usd_per_l     0.073176
unemployment             0.565000
dtype: float64

Update the aggregation functions called by `.agg()`: include `iqr` and `np.median` in that order.

In [12]:
sales[["temperature_c", "fuel_price_usd_per_l", "unemployment"]].agg([iqr, np.median])

/tmp/ipykernel_1586/3945324015.py:1: FutureWarning: The provided callable <function median at 0x7f4d20bc36d0> is currently using Series.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  sales[["temperature_c", "fuel_price_usd_per_l", "unemployment"]].agg([iqr, np.median])
/tmp/ipykernel_1586/3945324015.py:1: FutureWarning: The provided callable <function median at 0x7f4d20bc36d0> is currently using Series.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  sales[["temperature_c", "fuel_price_usd_per_l", "unemployment"]].agg([iqr, np.median])
/tmp/ipykernel_1586/3945324015.py:1: FutureWarning: The provided callable <function median at 0x7f4d20bc36d0> is currently using Series.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "me

,temperature_c,fuel_price_usd_per_l,unemployment
iqr,16.583333,0.073176,0.565
median,16.966667,0.743381,8.099


<a id="cum-stats"></a>
## 1.4 Cumulative statistics<br>
Cumulative statistics can also be helpful in tracking summary statistics over time. In this exercise, you'll calculate the cumulative sum and cumulative max of a department's weekly sales, which will allow you to identify what the total sales were so far as well as what the highest weekly sales were so far.

A DataFrame called `sales_1_1` has been created for you, which contains the sales data for department 1 of store 1. `pandas` is loaded as `pd`.

In [13]:
sales_1_1 = sales[(sales["department"] == 1) & (sales["store"] == 1)]

Sort the rows of `sales_1_1` by the `date` column in ascending order.

In [14]:
sales_1_1.sort_values("date", ascending=True)

,store,type,department,date,weekly_sales,is_holiday,temperature_c,fuel_price_usd_per_l,unemployment
0,1,A,1,2010-02-05,24924.50,False,5.727778,0.679451,8.106
1,1,A,1,2010-03-05,21827.90,False,8.055556,0.693452,8.106
2,1,A,1,2010-04-02,57258.43,False,16.816667,0.718284,7.808
3,1,A,1,2010-05-07,17413.94,False,22.527778,0.748928,7.808
4,1,A,1,2010-06-04,17558.09,False,27.050000,0.714586,7.808
5,1,A,1,2010-07-02,16333.14,False,27.172222,0.705076,7.787
6,1,A,1,2010-08-06,17508.41,False,30.644444,0.693980,7.787
7,1,A,1,2010-09-03,16241.78,False,27.338889,0.680772,7.787
8,1,A,1,2010-10-01,20094.19,False,22.161111,0.687640,7.838
9,1,A,1,2010-11-05,34238.88,False,14.855556,0.710359,7.838


Get the cumulative sum of `weekly_sales` and add it as a new column of `sales_1_1` called `cum_weekly_sales`.

In [15]:
sales_1_1["cum_weekly_sales"] = sales_1_1["weekly_sales"].cumsum()

/tmp/ipykernel_1586/1746300685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_1_1["cum_weekly_sales"] = sales_1_1["weekly_sales"].cumsum()


Get the cumulative maximum of `weekly_sales`, and add it as a column called `cum_max_sales`.

In [16]:
sales_1_1["cum_max_sales"] = sales_1_1["weekly_sales"].cummax()

/tmp/ipykernel_1586/4275078607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_1_1["cum_max_sales"] = sales_1_1["weekly_sales"].cummax()


Print the `date`, `weekly_sales`, `cum_weekly_sales`, and `cum_max_sales` columns.

In [17]:
sales_1_1[["date", "weekly_sales", "cum_weekly_sales", "cum_max_sales"]]

,date,weekly_sales,cum_weekly_sales,cum_max_sales
0,2010-02-05,24924.50,24924.50,24924.50
1,2010-03-05,21827.90,46752.40,24924.50
2,2010-04-02,57258.43,104010.83,57258.43
3,2010-05-07,17413.94,121424.77,57258.43
4,2010-06-04,17558.09,138982.86,57258.43
5,2010-07-02,16333.14,155316.00,57258.43
6,2010-08-06,17508.41,172824.41,57258.43
7,2010-09-03,16241.78,189066.19,57258.43
8,2010-10-01,20094.19,209160.38,57258.43
9,2010-11-05,34238.88,243399.26,57258.43


<a id="count"></a>
# __2 Counting__
<a id="drop-dup"></a>
## 2.1 Dropping duplicates
Removing duplicates is an essential skill to get accurate counts because often, you don't want to count the same thing multiple times. In this exercise, you'll create some new DataFrames using unique values from `sales`.

Remove rows of sales with duplicate pairs of `store` and `type` and save as `store_types` and print the head.

In [18]:
store_types = sales.drop_duplicates(["store", "type"])
store_types.head()

,store,type,department,date,weekly_sales,is_holiday,temperature_c,fuel_price_usd_per_l,unemployment
0,1,A,1,2010-02-05,24924.50,False,5.727778,0.679451,8.106
901,2,A,1,2010-02-05,35034.06,False,4.550000,0.679451,8.324
1798,4,A,1,2010-02-05,38724.42,False,6.533333,0.686319,8.623
2699,6,A,1,2010-02-05,25619.00,False,4.683333,0.679451,7.259
3593,10,B,1,2010-02-05,40212.84,False,12.411111,0.782478,9.765


Remove rows of `sales` with duplicate pairs of `store` and `department` and save as `store_depts` and print the head.

In [19]:
store_depts = sales.drop_duplicates(["store", "department"])
store_depts.head()

,store,type,department,date,weekly_sales,is_holiday,temperature_c,fuel_price_usd_per_l,unemployment
0,1,A,1,2010-02-05,24924.50,False,5.727778,0.679451,8.106
12,1,A,2,2010-02-05,50605.27,False,5.727778,0.679451,8.106
24,1,A,3,2010-02-05,13740.12,False,5.727778,0.679451,8.106
36,1,A,4,2010-02-05,39954.04,False,5.727778,0.679451,8.106
48,1,A,5,2010-02-05,32229.38,False,5.727778,0.679451,8.106


Subset the rows that are holiday weeks using the `is_holiday` column, and drop the duplicate `dates`, saving as `holiday_dates`.

In [20]:
holiday_dates = sales[sales["is_holiday"]].drop_duplicates("date")

Select the `date` column of `holiday_dates`, and print.

In [21]:
print(holiday_dates["date"])

498     2010-09-10
691     2011-11-25
2315    2010-02-12
6735    2012-09-07
6810    2010-12-31
6815    2012-02-10
6820    2011-09-09
Name: date, dtype: object


<a id="count-categ"></a>
## 2.2 Counting categorical variables<br>
Counting is a great way to get an overview of your data and to spot curiosities that you might not notice otherwise. In this exercise, you'll count the number of each type of store and the number of each department number using the DataFrames you created in the previous exercise:

\# Drop duplicate store/type combinations <br>
`store_types = sales.drop_duplicates(subset=["store", "type"])`<br><br>
\# Drop duplicate store/department combinations<br>
`store_depts = sales.drop_duplicates(subset=["store", "department"])`

Count the number of stores of each store `type` in `store_types`.

In [22]:
store_counts = store_types["type"].value_counts()
store_counts

type
A    11
B     1
Name: count, dtype: int64

Count the proportion of stores of each store `type` in `store_types`.

In [23]:
store_props = store_types["type"].value_counts(normalize=True)
store_props

type
A    0.916667
B    0.083333
Name: proportion, dtype: float64

Count the number of different `department`s in `store_depts`, sorting the counts in descending order.

In [24]:
dept_counts_sorted = store_depts["department"].value_counts(sort=True)
dept_counts_sorted

department
1     12
55    12
72    12
71    12
67    12
      ..
37    10
48     8
50     6
39     4
43     2
Name: count, Length: 80, dtype: int64

Count the proportion of different `department`s in `store_depts`, sorting the proportions in descending order.

In [25]:
dept_props_sorted = store_depts["department"].value_counts(sort=True, normalize=True)
dept_props_sorted

department
1     0.012917
55    0.012917
72    0.012917
71    0.012917
67    0.012917
        ...   
37    0.010764
48    0.008611
50    0.006459
39    0.004306
43    0.002153
Name: proportion, Length: 80, dtype: float64

<a id="grp-stat"></a>
# 3 Grouped Summary Statistics
<a id="percent"></a>
## 3.1 What percent of sales occurred at each store type?
While .groupby() is useful, you can calculate grouped summary statistics without it.

Walmart distinguishes three types of stores: "supercenters," "discount stores," and "neighborhood markets," encoded in this dataset as type "A," "B," and "C." In this exercise, you'll calculate the total sales made at each store type, without using .groupby(). You can then use these numbers to see what proportion of Walmart's total sales were made at each type.

Calculate the total weekly_sales over the whole dataset.

In [26]:
# Calc total weekly sales
sales_all = sales["weekly_sales"].sum()

Subset for type "A" stores, and calculate their total weekly sales.

In [27]:

# Subset for type A stores, calc total weekly sales
sales_A = sales[sales["type"] == "A"]["weekly_sales"].sum()

Do the same for type "B" and type "C" stores.

In [28]:
# Subset for type B stores, calc total weekly sales
sales_B = sales[sales["type"] == "B"]["weekly_sales"].sum()
# Subset for type C stores, calc total weekly sales
sales_C = sales[sales["type"] == "C"]["weekly_sales"].sum()

Combine the A/B/C results into a list, and divide by sales_all to get the proportion of sales by type.

In [29]:
# Get proportion for each type
sales_propn_by_type = [sales_A, sales_B, sales_C] / sales["weekly_sales"].sum()
print(sales_propn_by_type)

[0.9097747 0.0902253 0.       ]


<a id="grp"></a>
## 3.2 Calculations with .groupby()
The .groupby() method makes life much easier. In this exercise, you'll perform the same calculations as last time, except you'll use the .groupby() method. You'll also perform calculations on data grouped by two variables to see if sales differ by store type depending on if it's a holiday week or not.

Group sales by "type", take the sum of "weekly_sales", and store as sales_by_type.

In [40]:
sales_by_type = sales.groupby("type")["weekly_sales"].sum()
sales_by_type

type
A    2.337163e+08
B    2.317840e+07
Name: weekly_sales, dtype: float64

Calculate the proportion of sales at each store type by dividing by the sum of sales_by_type. Assign to sales_propn_by_type.

In [44]:
sales_propn_by_type = sales_by_type / sum(sales["weekly_sales"])
sales_propn_by_type

type
A    0.909775
B    0.090225
Name: weekly_sales, dtype: float64

Group sales by "type" and "is_holiday", take the sum of weekly_sales, and store as sales_by_type_is_holiday.

In [45]:
# Group by type and is_holiday; calc total weekly sales
sales_by_type_is_holiday = sales.groupby(["type", "is_holiday"])["weekly_sales"].sum()
sales_by_type_is_holiday

type  is_holiday
A     False         2.336927e+08
      True          2.360181e+04
B     False         2.317678e+07
      True          1.621410e+03
Name: weekly_sales, dtype: float64

<a id="multi"></a>
## 3.3 Multiple grouped summaries
Earlier in this chapter, you saw that the .agg() method is useful to compute multiple statistics on multiple variables. It also works with grouped data. NumPy, which is imported as np, has many different summary statistics functions, including: np.min, np.max, np.mean, and np.median.

In [48]:
import numpy as np

Get the min, max, mean, and median of weekly_sales for each store type using .groupby() and .agg(). Store this as sales_stats. Make sure to use numpy functions!

In [50]:
sales_stats = sales.groupby("type")["weekly_sales"].agg([np.min, np.max, np.mean, np.median])
sales_stats

/tmp/ipykernel_1586/4277130208.py:1: FutureWarning: The provided callable <function min at 0x7f4d3017db40> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  sales_stats = sales.groupby("type")["weekly_sales"].agg([np.min, np.max, np.mean, np.median])
/tmp/ipykernel_1586/4277130208.py:1: FutureWarning: The provided callable <function max at 0x7f4d3017da20> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  sales_stats = sales.groupby("type")["weekly_sales"].agg([np.min, np.max, np.mean, np.median])
/tmp/ipykernel_1586/4277130208.py:1: FutureWarning: The provided callable <function mean at 0x7f4d3017e320> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass t

,min,max,mean,median
type,,,,
A,-1098.0,293966.05,23674.667242,11943.92
B,-798.0,232558.51,25696.678370,13336.08


Get the min, max, mean, and median of unemployment and fuel_price_usd_per_l for each store type. Store this as unemp_fuel_stats.

In [58]:
unemp_fuel_stats = sales.groupby("type")[["unemployment", "fuel_price_usd_per_l"]].agg([np.min, np.max, np.mean, np.median])
unemp_fuel_stats

/tmp/ipykernel_1586/252427559.py:1: FutureWarning: The provided callable <function min at 0x7f4d3017db40> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  unemp_fuel_stats = sales.groupby("type")[["unemployment", "fuel_price_usd_per_l"]].agg([np.min, np.max, np.mean, np.median])
/tmp/ipykernel_1586/252427559.py:1: FutureWarning: The provided callable <function max at 0x7f4d3017da20> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  unemp_fuel_stats = sales.groupby("type")[["unemployment", "fuel_price_usd_per_l"]].agg([np.min, np.max, np.mean, np.median])
/tmp/ipykernel_1586/252427559.py:1: FutureWarning: The provided callable <function mean at 0x7f4d3017e320> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided call

unemployment                         fuel_price_usd_per_l            \
              min    max      mean median                  min       max   
type                                                                       
A           3.879  8.992  7.972611  8.067             0.664129  1.107410   
B           7.170  9.765  9.279323  9.199             0.760023  1.107674   

                          
          mean    median  
type                      
A     0.744619  0.735455  
B     0.805858  0.803348

<a id="pivo"></a>
# 4 Pivot tables
<a id="pivo-one"></a>
## 4.1 Pivoting on one variable
Pivot tables are the standard way of aggregating data in spreadsheets.

In pandas, pivot tables are essentially another way of performing grouped calculations. That is, the .pivot_table() method is an alternative to .groupby().

In this exercise, you'll perform calculations using .pivot_table() to replicate the calculations you performed in the last lesson using .groupby().

Get the mean weekly_sales by type using .pivot_table() and store as mean_sales_by_type.

In [59]:
mean_sales_by_type = sales.pivot_table(index="type", values="weekly_sales")
mean_sales_by_type

,weekly_sales
type,
A,23674.667242
B,25696.678370


Get the mean and median (using NumPy functions) of weekly_sales by type using .pivot_table() and store as mean_med_sales_by_type.

In [66]:
mean_med_sales_by_type = sales.pivot_table(index="type", values="weekly_sales", aggfunc=[np.mean, np.median])
#mean_med_sales_by_type = sales.pivot_table(index="type", values="weekly_sales", aggfunc=["mean", "median"]) # more modern way

mean_med_sales_by_type

/tmp/ipykernel_1586/4157367354.py:1: FutureWarning: The provided callable <function mean at 0x7f4d3017e320> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  mean_med_sales_by_type = sales.pivot_table(index="type", values="weekly_sales", aggfunc=[np.mean, np.median])
/tmp/ipykernel_1586/4157367354.py:1: FutureWarning: The provided callable <function median at 0x7f4d20bc36d0> is currently using DataFrameGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  mean_med_sales_by_type = sales.pivot_table(index="type", values="weekly_sales", aggfunc=[np.mean, np.median])


,mean,median
,weekly_sales,weekly_sales
type,,
A,23674.667242,11943.92
B,25696.678370,13336.08


Get the mean of weekly_sales by type and is_holiday using .pivot_table() and store as mean_sales_by_type_holiday.

In [67]:
mean_sales_by_type_holiday = sales.pivot_table(index="type", values="weekly_sales", columns="is_holiday")
mean_sales_by_type_holiday

is_holiday,False,True
type,,
A,23768.583523,590.04525
B,25751.980533,810.70500


## 4.2 Fill in missing values and sum values with pivot tables
The .pivot_table() method has several useful arguments, including fill_value and margins.

fill_value replaces missing values with a real value (known as imputation). What to replace missing values with is a topic big enough to have its own course (Dealing with Missing Data in Python), but the simplest thing to do is to substitute a dummy value.
margins is a shortcut for when you pivoted by two variables, but also wanted to pivot by each of those variables separately: it gives the row and column totals of the pivot table contents.
In this exercise, you'll practice using these arguments to up your pivot table skills, which will help you crunch numbers more efficiently!

Print the mean weekly_sales by department and type, filling in any missing values with 0.

In [68]:
sales.pivot_table(index="department", columns="type", values="weekly_sales", fill_value=0)

type,A,B
department,,
1,30961.725379,44050.626667
2,67600.158788,112958.526667
3,17160.002955,30580.655000
4,44285.399091,51219.654167
5,34821.011364,63236.875000
...,...,...
95,123933.787121,77082.102500
96,21367.042857,9528.538333
97,28471.266970,5828.873333


Print the mean weekly_sales by department and type, filling in any missing values with 0 and summing all rows and columns.

In [73]:
# note -> margin's mean doesn't include filled_value
sales.pivot_table(index="department", columns="type", values="weekly_sales", fill_value=0, margins=True)

type,A,B,All
department,,,
1,30961.725379,44050.626667,32052.467153
2,67600.158788,112958.526667,71380.022778
3,17160.002955,30580.655000,18278.390625
4,44285.399091,51219.654167,44863.253681
5,34821.011364,63236.875000,37189.000000
...,...,...,...
96,21367.042857,9528.538333,20337.607681
97,28471.266970,5828.873333,26584.400833
98,12875.423182,217.428333,11820.590278
